<a target="_blank" href="https://colab.research.google.com/github/umanitoba-meagher-projects/public-experiments/blob/main/jupyter-notebooks/Create%20Synthetic%20Images/prompt-generator.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
"""
Author: A.V. Ronquillo
Date: July 25, 2024

## Purpose: This notebook is a detailed guide containing modules for a Python script that is tasked with prompt generation.
## Note: Note: The author generated this text in part with GPT-4,
OpenAI’s large-scale language-generation model. Upon generating
draft code, the author reviewed, edited, and revised the code
to their own liking and takes ultimate responsibility for
the content of this code.

"""

# Introduction

This notebook explains methods for generating text prompts for image synthesis, tailored for use with models such as Stable Diffusion. The notebook is a guide through the process of constructing a large, diverse set of prompts that describe various animal activities and positions.

# Critical Uses & Adaptability

## What the Notebooks Can Be Used For:

- **Dataset Exploration:** The notebook can be used to systematically generate and examine a wide range of textual prompts, supporting the exploration of possible image scenarios for dataset creation or augmentation. This is particularly relevant for tasks that require controlled variation in image content.

- **Educational Purposes & Demonstrations:** The notebook serves as a resource for understanding how Python scripts and code modules can be employed in the context of image-related machine learning tasks.

- **Feature Extraction:** By generating prompts that describe specific activities and positions, the notebook can assist in the creation of datasets that facilitate feature extraction or analysis in downstream machine learning models. The structure of the prompts can be tailored to highlight particular attributes or behaviors.

## How the Notebook Can Be Adapted:

- **Integration with Spatial Design & Architectural Studies:** The methodology can be extended to generate prompts relevant to spatial arrangements, site analysis, or design scenarios. By modifying the activity and position variables, it's possible to adapt the workflow to describe spatial features, object placements, or environmental contexts in various datasets relevant to design.

- **Variables & Customization:** The notebook's variables, such as the lists of activities and positions (see the cell defining `activities` and `positions`) can be edited to reflect different subjects, actions, or spatial descriptors.

## Importing Necessary Libraries

`itertools` provides building blocks for constructing iterators for efficient looping. In this script, it is used to generate Cartesian products, which are all possible combinations of activities and positions. Importing `random` allows scripts to generate random numbers and shuffling data, in this instance, it is used to randomize the order of generated prompts.

The `os` library simply integrates settings that allows the script to interact with operating systems like directory creations and constructing file paths. The `requests` library enables HTTP requests for downloading data from the Borealis repository, and `zipfile` allows for handling compressed data files.

In [ ]:
import itertools
import random
import os
import requests
import zipfile

# Borealis Data Access & Define File Paths

The following code sets up access to the Borealis data repository. Borealis is a public data repository that hosts datasets without requiring authentication. The code includes functions to access dataset information, download files, and handle zip archives.

In [ ]:
# Borealis API configuration
BOREALIS_SERVER = "https://borealisdata.ca"

def get_public_dataset_info(persistent_id):
    """
    Get information about a public dataset
    """
    url = f"{BOREALIS_SERVER}/api/datasets/:persistentId/"
    params = {"persistentId": persistent_id}

    response = requests.get(url, params=params)

    if response.status_code == 200:
        dataset_info = response.json()
    else:
        print(f"Cannot access dataset: {response.status_code}")
        return None
    """
    Get a list of files in a public dataset
    """
    # Access the list of files from the dataset_info dictionary
    files_list = dataset_info['data']['latestVersion']['files']

    # Create an empty list to store file information
    file_info_list = []

    # Iterate through the files list and append file ID and filename to the list
    for file_info in files_list:
        file_id = file_info['dataFile']['id']
        filename = file_info['dataFile']['filename']
        file_info_list.append({"file_id": file_id, "filename": filename})

    return file_info_list

def download_public_file(file_id, save_path="./"):
    """
    Download a specific public file from a dataset by its file ID
    No authentication required
    """
    url = f"{BOREALIS_SERVER}/api/access/datafile/{file_id}"

    response = requests.get(url, stream=True)

    if response.status_code == 200:
        # Determine filename from headers or URL
        filename = None
        if "Content-Disposition" in response.headers:
            cd = response.headers["Content-Disposition"]
            # Try to extract filename from content disposition
            if "filename=" in cd:
                filename = cd.split("filename=")[1].strip('"')

        # Fallback to extracting from URL if header not available or malformed
        if not filename:
             filename = url.split("/")[-1]

        file_path = f"{save_path}/{filename}"

        with open(file_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

        print(f"✅ File downloaded to {file_path}")
        return file_path
    else:
        print(f"❌ Error {response.status_code}: File may be restricted or not found")
        return None

def is_zip_file(filepath):
    """
    Checks if a file is a valid zip file.
    """
    return zipfile.is_zipfile(filepath)

def unzip_file(filepath, extract_path="./"):
    """
    Unzips a zip file to a specified path and returns the name of the top-level extracted folder.
    Returns None if not a zip file or extraction fails.
    """
    if is_zip_file(filepath):
        try:
            with zipfile.ZipFile(filepath, 'r') as zip_ref:
                # Get the name of the top-level directory within the zip
                # Assumes there is a single top-level directory
                top_level_folder = None
                for file_info in zip_ref.infolist():
                    parts = file_info.filename.split('/')
                    if parts[0] and len(parts) > 1:
                        top_level_folder = parts[0]
                        break # Assuming the first entry gives the top-level folder

                zip_ref.extractall(extract_path)
                print(f"✅ Successfully unzipped {filepath} to {extract_path}")
                return top_level_folder

        except Exception as e:
            print(f"❌ Error unzipping {filepath}: {e}")
            return None
    else:
        print(f"ℹ️ {filepath} is not a valid zip file.")
        return None

# Initialize Borealis dataset access
public_doi = "doi:10.5683/SP3/H3HGWF"
print("Borealis dataset initialized for animal notebook data.")

The `output_folder` defines the local path where the output file will be saved. Since we're using Borealis data access instead of Google Drive, files are stored locally in the current working directory.
`os.makedirs(output_folder, exist_ok=True)` creates the directory specified by output folder. If the directory already exists, the `exist_ok=True` parameter prevents an error from being raised, ensuring that the folder structure is in place before attempting to save the file. The output path is then combined with the filename `squirrel_prompts7.txt` by using `os.path.join`. However, in this instance, the filename can be changed for customization to `your_file_name.txt` This creates a full path for the output file, ensuring that it is saved in the correct location.

In [ ]:
# Define the output folder and file path
output_folder = './output_prompts'
os.makedirs(output_folder, exist_ok=True)
output_file_path = os.path.join(output_folder, "squirrel_prompts7.txt")    # Can be changed to "your_file_name_.txt"

# Generating Prompts through Sentence Parameters

The process of generating the prompts is executed in this module. The sentence parameters revolve around different conditions. Firstly, the snippet of code uses an `activities` arguement for the creation of a list of various verbs describing different actions or states an animal might be engaged in. For example, useful verbs include "walking", "foraging", "running", or "looking around". Secondly, the `positions` arguement creates a list describing different possible positions or scaling factors for the animal within the image. These descriptions are used to specify how the animal should be sized or positioned. Lastly, these conditions will be used to integrate into a `base_prompt`. This base sentence is the initial part of each prompt, providing a consistent context for the description of the animal. In this instance it describes a "realistic photograph" of a "singular squirrel" without any background.

In [ ]:
def generate_all_prompts():
    # Fox activities (verbs)
    activities = ["walking", "foraging", "up close", "standing", "running", "sitting", "jogging", "looking around", "looking away", "lingering"]

    # Positions within the image
    positions = ["SCALED on the right side of the overall frame of the empty image", "SCALED on the left side of the overall frame of the empty image", "SCALED in the center of the overall frame of the empty image",
                 "SCALED perspectivally in the foreground as a bigger squirrel", "SCALED perspectivally in the background as a smaller squirrel", "SCALED perspectivally in the midground as a medium size squirrel"]

    # Base prompt
    base_prompt = "A realistic photograph of one singular squirrel by itself with no background image. This cutout of the squirrel is"

    # Generate all possible combinations of activities and positions
    combinations = list(itertools.product(activities, positions))

    # List to store the generated prompts
    prompts = []

    for activity, position in combinations:
        prompt = f"{base_prompt} {activity} {position}."
        prompts.append(prompt)

    # Shuffle prompts to add randomness if desired
    random.shuffle(prompts)

    return prompts

This snippet of code recalls the `itertools.product` to use words specified in the positions and the activities arguements, creating every possible combination and a comprehensive list of prompt variations. The `prompts` argument intializes an empty list (`[]`) to store each generated prompt. Each combination of an activity and position is then iterated over, creating a full prompt string by appending the activity and position to the  `base_prompt`.

The `random.shuffle(prompts)` sequences and shuffles through the list of prompts to ensure they are not in a predictable order to add a degree of randomness. The list of all the generated prompts are then returned to be implemented into the next part of the script.



# Writing each Prompt to a Text File

This final module explains the main block of code that executes on the writing of the generated prompts. The number (`num_prompts`) of prompts you want can be specified for image generation, which can be changed depending on how many prompts is the desired amount. `generate_all_prompts()` then calls the function to generate the list of prompts as the `open` arguement is used with the `output_file_path` to open the specified text file in write mode. This also ensures that the text file is properly closed after writing even if an error occurs.

Each prompt in the set is iterated through and written through the `file.write` function. The `(f"{prompt}\n")` writes each prompt to the file, adding a newline character (\n) after each prompt to ensure each one is on a new line. It then saves the `num_prompts` to an output file path and prints a confirmation message with an indication of how many prompts were written as a total of all the possible combinations of the base prompt.



In [ ]:
if __name__ == "__main__":
    num_prompts = 400  # Total number of prompts
    prompts = generate_all_prompts()

    # Write the generated prompts to an output text file
    with open(output_file_path, "w") as file:
        for prompt in prompts[:num_prompts]:
            file.write(f"{prompt}\n")

    print(f"{num_prompts} prompts have been written to '{output_file_path}'")